# SET

How many cards are there? There are four features (color, shape, shading, and number of figures) on each card, and each feature can take one of three values.  So the number of cards is:

In [10]:
3 ** 4

81

How many sets are there?

In [11]:
81 * 80 * 1 / 6

1080.0

How many sets does each card participate in?  We need to look at the number of sets (1080) and the number of cards in a set (3), divided by the number of cards (81):

In [23]:
1080 * 3 / 81

40.0

Note that each *pair* of cards participates in exactly one set.

How many layouts of 12 cards are there? The answer is (81 choose 12):

In [43]:
from math import factorial as fact

def C(n, k): 
    "Number of ways of choosing k things from n things."
    return fact(n) / fact(n-k) / fact(k)

C(81, 12)

70724320184700.0

That's a lot of digits; hard to read. This should help:

In [42]:
M = 10 ** 6  # Million
T = 10 ** 12 # Trillion

C(81, 12) / T

70.7243201847

70 trillion layouts. Of those some will have 6 sets, some more, some less.

In a layout of 12 cards, how many triples (that could potentially be a set) are there?

In [44]:
C(12, 3)

220.0

So, what we're looking for is when exactly 6 of these are sets, and the other 214 are not.

In [31]:
C(1080, 6)  / T

2173.5413336637

In [2]:
from itertools import product

feature = (1, 2, 3)

Card = tuple

cards = set(product(feature, feature, feature, feature))
len(cards)

81

In [3]:
def match(card1, card2):
    return Card(match_feature(card1[i], card2[i]) 
                for i in (0, 1, 2, 3))

def match_feature(f1, f2):
    return f1 if f1 == f2 else 6 - f1 - f2

In [15]:
match((1, 2, 3, 3),
      (1, 1, 2, 3))

(1, 3, 1, 3)

2173 trillion; even worse.

How many layouts are there where:
- The first six cards form no sets
- The last six cards each form a set?

In [38]:
81 * 80 * (79 - 1) * (78 - 3) * (77 - 6) * (76 - 10) / fact(6) / M

246.7179

In [40]:
15 * 21 * 28 * 36 * 45 * 55 / fact(6) / M

1.091475

In [4]:
import random
import collections 
import itertools 

"""
Game of Set                (Peter Norvig 2010-2015)

How often do sets appear when we deal an array of cards?
How often in the course of playing out the game?

Here are the data types we will use:

    card:    A string, such as '3R=0', meaning "three red striped ovals".
    deck:    A list of cards, initially of length 81.
    layout:  A list of cards, initially of length 12.
    set:     A tuple of 3 cards.
    Tallies: A dict: {12: {True: 33, False: 1}}} means a layout of size 12
             tallied 33 sets and 1 non-set.
"""

#### Cards, dealing cards, and defining the notion of sets.

CARDS = [number + color + shade + symbol 
         for number in '123' 
         for color  in 'RGP' 
         for shade  in '@O=' 
         for symbol in '0SD']

def deal(n, deck): 
    "Deal n cards from the deck."
    return [deck.pop() for _ in range(n)]

def is_set(cards):
    "Are these 3 cards a set? No if any feature has 2 values."
    for f in range(4):
        values = {card[f] for card in cards}
        if len(values) == 2: 
            return False
    return True

def find_set(layout):
    "Return a set found from this layout, if there is one."
    for cards in itertools.combinations(layout, 3):
        if is_set(cards):
            return cards
    return ()

#### Tallying set:no-set ratio

def Tallies(): 
    "A data structure to keep track, for each size, the number of sets and no-sets."
    return collections.defaultdict(lambda: {True: 0, False: 0})

def tally(tallies, layout):
    "Record that a set was found or not found in a layout of given size; return the set."
    s = find_set(layout)
    tallies[len(layout)][bool(s)] += 1
    return s
            
#### Three experiments

def tally_initial_layout(N, sizes=(12, 15)):
    "Record tallies for N initial deals."
    tallies = Tallies()
    deck = list(CARDS)
    for deal in range(N):
        random.shuffle(deck)
        for size in sizes:
            tally(tallies, deck[:size])
    return tallies

def tally_initial_layout_no_prior_sets(N, sizes=(12, 15)):
    """Simulate N initial deals for each size, keeping tallies for Sets and NoSets,
    but only when there was no set with 3 fewer cards."""
    tallies = Tallies()
    deck = list(CARDS)
    for deal in range(N):
        random.shuffle(deck)
        for size in sizes:
            if not find_set(deck[:size-3]):
                tally(tallies, deck[:size])
    return tallies

def tally_game_play(N):
    "Record tallies for the play of N complete games."
    tallies = Tallies()
    for game in range(N):
        deck = list(CARDS)
        random.shuffle(deck)
        layout = deal(12, deck)
        while deck:
            s = tally(tallies, layout)
            # Pick up the cards in the set, if any
            for card in s: layout.remove(card)
            # Deal new cards
            if len(layout) < 12 or not s:
                layout += deal(3, deck)    
    return tallies

def experiments(N):
    show({12: [1, 33], 15: [1, 2500]}, 
         'the instruction booklet')
    show(tally_initial_layout(N), 
         'initial layout')
    show(tally_game_play(N // 25), 
         'game play')
    show(tally_initial_layout_no_prior_sets(N), 
         'initial layout, but no sets before dealing last 3 cards')


def show(tallies, label):
    "Print out the counts."
    print()
    print('Size |  Sets  | NoSets | Set:NoSet ratio for', label)
    print('-----+--------+--------+----------------')
    for size in sorted(tallies):
        y, n = tallies[size][True], tallies[size][False]
        ratio = ('inft' if n==0 else int(round(float(y)/n)))
        print('{:4d} |{:7,d} |{:7,d} | {:4}:1'
              .format(size, y, n, ratio))

def test():
    assert len(CARDS) == 81 == len(set(CARDS))
    assert is_set(('3R=O', '2R=S', '1R=D'))
    assert not is_set(('3R=0', '2R=S', '1R@D'))
    assert find_set(['1PO0', '2G=D', '3R=0', '2R=S', '1R=D']) == ('3R=0', '2R=S', '1R=D')
    assert not find_set(['1PO0', '2G=D', '3R=0', '2R=S', '1R@D'])
    photo = '2P=0 3P=D 2R=0 3GO0 2POD 3R@D 2RO0 2ROS 1P@S 2P@0 3ROS 2GOD 2P@D 1GOD 3GOS'.split()
    assert not find_set(photo)
    assert set(itertools.combinations([1, 2, 3, 4], 3)) == {(1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)}
    print('All tests pass.')

test()
experiments(100000)

All tests pass.

Size |  Sets  | NoSets | Set:NoSet ratio for the instruction booklet
-----+--------+--------+----------------
  12 |     33 |      1 |   33:1
  15 |  2,500 |      1 | 2500:1

Size |  Sets  | NoSets | Set:NoSet ratio for initial layout
-----+--------+--------+----------------
  12 | 96,844 |  3,156 |   31:1
  15 | 99,963 |     37 | 2702:1

Size |  Sets  | NoSets | Set:NoSet ratio for game play
-----+--------+--------+----------------
  12 | 86,065 |  5,871 |   15:1
  15 |  5,595 |     64 |   87:1
  18 |     57 |      0 | inft:1

Size |  Sets  | NoSets | Set:NoSet ratio for initial layout, but no sets before dealing last 3 cards
-----+--------+--------+----------------
  12 | 26,426 |  3,242 |    8:1
  15 |  3,207 |     35 |   92:1


In [5]:
experiments(10000)


Size |  Sets  | NoSets | Set:NoSet ratio for the instruction booklet
-----+--------+--------+----------------
  12 |     33 |      1 |   33:1
  15 |  2,500 |      1 | 2500:1

Size |  Sets  | NoSets | Set:NoSet ratio for initial layout
-----+--------+--------+----------------
  12 |  9,696 |    304 |   32:1
  15 |  9,995 |      5 | 1999:1

Size |  Sets  | NoSets | Set:NoSet ratio for game play
-----+--------+--------+----------------
  12 |  8,653 |    542 |   16:1
  15 |    513 |      5 |  103:1
  18 |      5 |      0 | inft:1

Size |  Sets  | NoSets | Set:NoSet ratio for initial layout, but no sets before dealing last 3 cards
-----+--------+--------+----------------
  12 |  2,630 |    294 |    9:1
  15 |    293 |      1 |  293:1
